# Lab Exercise 07
---

## Analysis techniques


### Correlation analyses

We are going to look at a few correlation analysis techniques that can be used in Python. These functions are implemented in numerous packages, so we will check out a few to get a taste for different python modules.

As for the correlation methods, there are 4 we will focus on here:
1. Pearson correlation 
2. Spearman correlation
3. Kendall rank correlation (or Kendalls' tau)
4. Point biserial correlation

Each of these tests is used in different scenarios; however, this is not a stats class, so we will not be diving too deeply into the theory that is covered in a more formal stats class. I will briefly describe each and explain when it is appropriate to use.

**Pearson correlation** is used to calculate linear relationship between two continuous variables. The relationship is defined by the "correlation coefficient" that is calculated, along with the p-value. The correlation coefficient can fall within the range -1 to +1, with -1 being a perfect negative relationship (left), +1 being a perfect positive relationship (right), and 0 being weak or no relationship (middle)

![corr](img/pearson_corr.png)

To use this method, you must ensure that you satisfy all assumptions. These include:
1. Level of measurement: must be interval or ratio (continuous variable)
2. Normality: Both variables must have roughly normal distributions
3. Linear relationship: The two variables have a linear relationship as opposed to a non-linear relationship.
4. Paired data: For each measurement for one variable and paired value in the other variable must exist.
5. No outliers: check for extreme outliers present in the two variables.

We know some of these assumptions are true by understanding our data, i.e., each patient has a bmi and age, so they are paired data. We can check the other assumptions pretty quickly visually using some plotting techniques.

First, let's load up our dataset in to a dataframe.

In [ ]:
import pandas as pd

df = pd.read_csv("stroke_data.csv")
df.dropna(inplace=True)
df.info()

Let's check if the two variables (age and bmi) have "roughly" a linear relationship. We can check this visually using `relplot` in seaborn.

In [ ]:
import seaborn as sns
sns.relplot(df, x='age', y='bmi')

You can tell, it is not exactly linear, but we will proceed regardless and chec kthe other assumptions.

Next, let us check if each variable is nromally distributed. We will do this by using the `displot` function in seaborn.

In [ ]:
sns.displot(df, x='age')

This doesn't look bad...somewhat normal. Using the kernal density estimation (kde) may be more helpful in visualizing whether the plot is normal.

In [ ]:
sns.displot(df, x='age', kind='kde')

In [ ]:
sns.displot(norm)

We see a bit better from the kde that the distribution might have some heavy tails.

We can use another method to determine normality. This is called the Q-Q plot or quantile-quantile plot, which plots the calculated quantiles for a given variable against the theoretical quantiles. If normal, the plotted points should fall along the diagonal line.

We will leverage the function built in `statsmodel` called `qqplot` as shown below. The line to fit against will be `"s"` which means the standardized line with respect to the sample data.

In [ ]:
import statsmodels.api as sm

sm.qqplot(df.age, line='s', fit=True)

Now we are pretty confident that the age variable is not normally distributed. There are ways to account for this, e.g., taking the log of the values, etc. Here we will proceed for the sake of learning the other assumptions and running the test.

Let's check the bmi variable quickly.

In [ ]:
sns.displot(df, x='bmi')

In [ ]:
sns.displot(df, x='bmi', kind='kde')

In [ ]:
sm.qqplot(df.bmi, line='s', fit=True)

The last assumption we will check is the presence of extreme outliers in the data. A quick visual check is using a boxplot to see if any values fall outside the "min" or "max" of the whiskers.

We will use the `catplot` function in seaborn to plot these boxplots.

In [ ]:
sns.catplot(df, x='bmi', kind='box')

As you can clearly see there are quite a few outliers in the bmi variable, which we partially saw with the tail in the histogram plot.

If we are concerned about these outliers, we can always remove them with a fairly simple line of code using pandas and `numpy` and the `stats` subpackage within `scipy`. We will calculate the absolute value of the z score for each of the values in the bmi variable and only keep those that are less than 3 (within the 99.7% of the data).

In [ ]:
import numpy as np
from scipy import stats

df = df[(np.abs(stats.zscore(df['bmi'])) < 3)]

What does the boxplot look like now without those outliers?

In [ ]:
sns.catplot(df, x='bmi', kind='box')

Fewer outliers, especially those more extreme ones. This can be a way of handling this assumption violation, if it makes sense to do so.

Let's check if there are any outliers for the age variable.

In [ ]:
sns.catplot(df, x='age', kind='box')

Nope. Looks like we are good with the age variable and the "no outliers" assumption.

Even though we have violated at least one of the assumptions above, we will proceed with running the Pearson correlation test on these two variables.

To do so, we will again use the `stats` subpackage from `scipy`, specifically the `pearsonr` function. This function takes in the two set of data (age and bmi variables) and outputs a list containing two values: the correlation (as described above) and the p-value to determine the significance of the result.

In [ ]:
corr, p = stats.pearsonr(df['age'], df['bmi'])
print(f"Pearson correlation = {corr} and p-value = {p}")

If we ignore the fact that we violated the assumptions, the above results would tell us that the associaiton between the two variables is signfiicant (p<<<0.05), but it is a weak association (0.37).

Returning to the idea that we vioalted the assumptions for the Pearson correlation, what other methods can be used to determine correlation for data that fails any of the assumptions?

**Spearman rank correlation** is a non-parametric method used to calculate monotonic relationship between two variables. Also falls between -1 and +1, but is not directly concerned with the lienar nature of the relationship between two variables. This means Spearman correlation is less restrictive than pearson.

We can use this test to determine if the age varaible has a monotonic relationship with bmi.

This function can also be found in `scipy` and is calle `spearmanr`. The output is simialr to Pearson as it generated a list that contains the correlation value (rho) and the p-value.

In [ ]:
corr, p = stats.spearmanr(df['age'], df['bmi'])
print(f"Spearman correlation (rho) = {corr} and p-value = {p}")

As we saw with the Pearson correlation results, there is a weak relationship between these two variables.

**Kendall's tau** is yet another non-parametric correlation method that can be used when your data violates a parametric method like Pearson. This is very similar to Spearman in how the results are interpreted, measures the strength of relationship between two variables.

The `scipy` function `kendalltau` will calcualte the tau statistic (correlation coefficient) and the p-value. 

In [ ]:
corr, p = stats.kendalltau(df['age'], df['bmi'])
print(f"Kendall's tau = {corr} and p-value = {p}")

What if we are interested in assessing the correlation between a categorical and continuous variable? To do this we 
**Point biserial correlation**. Like Pearson, Point bierial corelation has a few assumptions that must be met. These include: 
1. Continuous and Binary
2. Normally Distributed
3. No Outliers
4. Equal Variances

We have already seen how to check the first three, to check the fourth we can visually inspect by plotting two histograms colored by the binary variable.

In [ ]:
sns.displot(df,x=df['bmi'],hue=df['heart_disease'])

However, if we want to be more quantitative in our assessment we can calculater the standard deviation of the bmi variable with respect to the heart_disease variable.

In [ ]:
print(df.loc[(df['heart_disease']==0),'bmi'].std())
print(df.loc[(df['heart_disease']==1),'bmi'].std())

If we do not violate any of the assumptions, we can use the point biserial correlation to assess the strength of relationship between the continuous and categorical variable. Here we use the `scipy` package again with the `pointbiserialr` function.

In [ ]:
corr, p = stats.pointbiserialr(df['age'], df['stroke'])
print(f"Point biserial correlation = {corr} and p-value = {p}")

Despite the relationship being significant, we see the strength of the relationship is weak.

If our data violates the assumptions set forth by the Point biserial correlation, then we would have to look into other methods that are less restrictive. One such method is **logistic regression**. We will discuss this method next week!

## Statistical hypothesis tests

Sticking with the trend of asssessing the relationship between two variables, the statistical test we will focus on is the **chi squared test**. This test determines if there is a statistical difference between the expected and observed frequencies for categories in a contigency table (cross table). Observed frequencies are those that exist in the data, whereas expected frequencies are calculated from the data in the table.

Chi square assumptions include:
1. Both variables are categorical.
2. All observations are independent.
3. Cells in the contingency table are mutually exclusive.
4. Expected value of cells should be 5 or greater in at least 80% of cells.

Let's first create a cross table of the two variables we are interested in testing: smoking_status and heart_disease.

In [ ]:
#data = pd.crosstab(df['smoking_status'], df['heart_disease'])
data = pd.crosstab(df['heart_disease'],df['smoking_status'])

Next, let's use the `chi2_contingency` function in `scipy.stats` to calulate the expected frequnencies, chi squared test statistic, and the p-value.

In [ ]:
chisq, p, dof, expected = stats.chi2_contingency(data)
print(f"Chi square test statisic = {chisq} and p-value = {p}")

Using these results, we can reject the null hypothesis and state that there is a difference between the categories.

We can further understand this relationship by assessing the effect size (strength of relationship) between heart_disease and smoking_status. To do so we will use **Cramer's V**, which is calculated by taking the square root of the chi squared statistic divided by the sample size and the minimum dimension minus (fewest number of categories between the two variables). The result falls between 0 and +1.

In [ ]:
result = np.sqrt((chisq/len(df)) / np.min([len(data.index),len(data.columns)]))

In [ ]:
print(f"Cramer's V = {result}")

Despite the difference in frequencies from the chi squared test, the Cramer's V denotes little to no relationship between the two variables.

---
# Graded Portion

Using the "water_potability.csv" dataset, answer the following questions in plain language and provide code, plots, and/or calculations to justify your answers. Remember to ***CLEAN*** the data first (remove missing values).

## Problem 01 (6 points)

Tell me if the following variables are normally distributed or not:
1. ph
2. Hardness
3. Solids

## Answer 01

Provide your answers here by modifying this cell:

1. 
2. 
3. 


In [ ]:
# Provide any accompanying code, plots, and calculations here and reference them in your answer.


## Problem 02 (6 points)

1. Which correlation method can be used to determine the relationship between ph and Hardness?
2. Using that method, what is the correlation and p-value for those two variables?

## Answer 02

Provide your answers here by modifying this cell:

1. 
2. 


In [ ]:
# Provide any accompanying code, plots, and calculations here and reference them in your answer.


## Problem 03 (8 points)

1. Which variable has the highest correlation with the variable Potability?
2. What is the correlation and p-value for that relationship?

## Answer 03

Provide your answers here by modifying this cell:

1. 
2. 


In [ ]:
# Provide any accompanying code, plots, and calculations here and reference them in your answer.
